# T-Test Table

In [5]:
import stata_setup

stata_setup.config('/Applications/StataNow', 'mp')

In [6]:
%%stata
/******************************************************************************************
* File: compare_whale_vs_smallholders_fullstats.do
* Goal: Small vs Whale for num, turnout:
*       Mean, SD, P10, Median, P90 (each group) + Diff (means) + t-stat (stars)
* Notes: # Holders -> 1 dp; Turnout -> 3 dp
******************************************************************************************/

clear all
set more off
set varabbrev off
global PROCESSED_DATA "processed_data"
global TABLES "tables"

* -------------------- LOAD DATA --------------------
import delimited "$PROCESSED_DATA/proposals_voter_user.csv", clear varn(1) case(lower)
capture destring whale* non_whale*, replace force

local metrics num_delegate turnout_delegate from_delegation_rate to_delegation_rate reason_rate vn_hhi timing_avg user

* -------------------- COLLECT RESULTS --------------------
tempfile results
postfile H str80(metric) ///
    double( ///
        ms_mean ms_sd ms_p10 ms_p50 ms_p90 ///
        mw_mean mw_sd mw_p10 mw_p50 mw_p90 ///
        diff_mean tstat pval ///
    ) str3(stars) using `results', replace

foreach m of local metrics {
    local whale whale_`m'
    local small non_whale_`m'

    * Small (non-whale) stats
    quietly summarize `small', detail
    local ms_mean = r(mean)
    local ms_sd   = r(sd)
    local ms_p10  = r(p10)
    local ms_p50  = r(p50)
    local ms_p90  = r(p90)

    * Whale stats
    quietly summarize `whale', detail
    local mw_mean = r(mean)
    local mw_sd   = r(sd)
    local mw_p10  = r(p10)
    local mw_p50  = r(p50)
    local mw_p90  = r(p90)

    * Difference in means with t-test
    quietly gen __diff = `small' - `whale'

    quietly reg __diff, vce(cluster space)

    local d  = _b[_cons]
    local tt = _b[_cons] / _se[_cons]
    local p  = 2 * ttail(e(df_r), abs(`tt'))

    drop __diff


    * Stars
    if (`p' < 0.01)      local stars "***"
    else if (`p' < 0.05) local stars "**"
    else if (`p' < 0.10) local stars "*"
    else                 local stars ""

    * Row label
    if "`m'" == "num_delegate"         local label "\$\textit{\# Holders}_{i}\$"
    else if "`m'" == "turnout_delegate"  local label "\$\textit{Turnout}_{i}\$"
    else if "`m'" == "from_delegation_rate"  local label "\$\textit{Delegator}_{i}\$"
    else if "`m'" == "to_delegation_rate"    local label "\$\textit{Delegatee}_{i}\$"
    else if "`m'" == "reason_rate"    local label "\$\textit{Reason}_{i}\$"
    else if "`m'" == "vn_hhi"    local label "\$\textit{HHI}_{i}^{vn}\$"
    else if "`m'" == "timing_avg"    local label "\$\textit{Timing}_{i}\$"
    else if "`m'" == "user"    local label "\$\textit{User}_{i}\$"
    else                      local label "`m'"

    post H ("`label'") ///
        (`ms_mean') (`ms_sd') (`ms_p10') (`ms_p50') (`ms_p90') ///
        (`mw_mean') (`mw_sd') (`mw_p10') (`mw_p50') (`mw_p90') ///
        (`d') (`tt') (`p') ("`stars'")
}
postclose H

* -------------------- LATEX EXPORT --------------------
use `results', clear

tempname fh
file open `fh' using "$TABLES/sum_whale_non_whale.tex", write replace

file write `fh' "{\small" _n
file write `fh' "\begin{tabular}{lcccccccccccc}" _n
file write `fh' "\toprule" _n
file write `fh' " & \multicolumn{5}{c}{Small Shareholder (Holding \$<\$ 5\%)} & \multicolumn{5}{c}{Whale (Holding \$\ge\$ 5\%)} & \multicolumn{2}{c}{Small - Whale} \\" _n
file write `fh' "\cmidrule(lr){2-6} \cmidrule(lr){7-11} \cmidrule(lr){12-13}" _n
file write `fh' " & Mean & Std. Dev. & P10 & Median & P90 & Mean & Std. Dev. & P10 & Median & P90 & Diff. & t-stat \\" _n
file write `fh' "\midrule" _n

local Nobs = _N
forvalues i = 1/`Nobs' {
    local metric = metric[`i']

    * Precision rules
    if "`metric'" == "\$\textit{\# Holders}_{i}\$" {
        local ms_mean_fmt : display %12.1f ms_mean[`i']
        local ms_sd_fmt   : display %12.1f ms_sd[`i']
        local ms_p10_fmt  : display %12.1f ms_p10[`i']
        local ms_p50_fmt  : display %12.1f ms_p50[`i']
        local ms_p90_fmt  : display %12.1f ms_p90[`i']

        local mw_mean_fmt : display %12.1f mw_mean[`i']
        local mw_sd_fmt   : display %12.1f mw_sd[`i']
        local mw_p10_fmt  : display %12.1f mw_p10[`i']
        local mw_p50_fmt  : display %12.1f mw_p50[`i']
        local mw_p90_fmt  : display %12.1f mw_p90[`i']

        local diff_fmt    : display %12.1f diff_mean[`i']
        local tstat_fmt   : display %12.1f tstat[`i']
    }
    else {
        * Fallback (3 dp)
        local ms_mean_fmt : display %12.4f ms_mean[`i']
        local ms_sd_fmt   : display %12.4f ms_sd[`i']
        local ms_p10_fmt  : display %12.4f ms_p10[`i']
        local ms_p50_fmt  : display %12.4f ms_p50[`i']
        local ms_p90_fmt  : display %12.4f ms_p90[`i']

        local mw_mean_fmt : display %12.4f mw_mean[`i']
        local mw_sd_fmt   : display %12.4f mw_sd[`i']
        local mw_p10_fmt  : display %12.4f mw_p10[`i']
        local mw_p50_fmt  : display %12.4f mw_p50[`i']
        local mw_p90_fmt  : display %12.4f mw_p90[`i']

        local diff_fmt    : display %12.4f diff_mean[`i']
        local tstat_fmt   : display %12.4f tstat[`i']
    }

    local st = stars[`i']
    file write `fh' ///
        "`metric' & `ms_mean_fmt' & `ms_sd_fmt' & `ms_p10_fmt' & `ms_p50_fmt' & `ms_p90_fmt' & " ///
        "`mw_mean_fmt' & `mw_sd_fmt' & `mw_p10_fmt' & `mw_p50_fmt' & `mw_p90_fmt' & " ///
        "`diff_fmt' & `tstat_fmt'`st' \\" _n
}

file write `fh' "\bottomrule" _n "\end{tabular}}" _n
file close `fh'



. /****************************************************************************
> **************
> * File: compare_whale_vs_smallholders_fullstats.do
> * Goal: Small vs Whale for num, turnout:
> *       Mean, SD, P10, Median, P90 (each group) + Diff (means) + t-stat (star
> s)
> * Notes: # Holders -> 1 dp; Turnout -> 3 dp
> *****************************************************************************
> *************/
. 
. clear all

. set more off

. set varabbrev off

. global PROCESSED_DATA "processed_data"

. global TABLES "tables"

. 
. * -------------------- LOAD DATA --------------------
. import delimited "$PROCESSED_DATA/proposals_voter_user.csv", clear varn(1) ca
> se(lower)
(encoding automatically selected: ISO-8859-1)
(39 vars, 2,681 obs)

. capture destring whale* non_whale*, replace force

. 
. local metrics num_delegate turnout_delegate from_delegation_rate to_delegatio
> n_rate reason_rate vn_hhi timing_avg user

. 
. * -------------------- COLLECT RESULTS -------------